In [1]:
# Given a path to MCMC datasets
# Recursively go through all directories and find data.json

from _utils import *

2024-09-30 14:36:06.326089: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 14:36:06.370534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mcmc_path = '/mnt/home/tudomlumleart/ceph/01_ChromatinEnsembleRefinement/chromatin-ensemble-refinement/MCMC_results/20240930_RunWeightMCMC_Tonia_PCA_2'

In [7]:
# Edit json files in mcmc_path
# List all items in the current directory
for item in tqdm(os.listdir(mcmc_path)):
    # Get the full path of the item
    full_path = os.path.join(mcmc_path, item)
    
    # Check if the item is a directory
    # And if the directory contains data.json
    # Check if 'data.json' exists in the directory
    file_path = os.path.join(full_path, 'data.json')
    if os.path.isfile(file_path):
        # Edit the json file
        with open(file_path, 'r') as f:
            data = json.load(f)

        json_obj = json.dumps(data, indent=4)
        with open(file_path, 'w') as f:
            f.write(json_obj)
            f.close()
        
        

  0%|          | 0/10 [00:00<?, ?it/s]

JSONDecodeError: Expecting ',' delimiter: line 29531144 column 26 (char 828711031)

In [5]:
# Edit json files in mcmc_path
# List all items in the current directory
for item in tqdm(os.listdir(mcmc_path)):
    # Get the full path of the item
    full_path = os.path.join(mcmc_path, item)
    
    # Check if the item is a directory
    # And if the directory contains data.json
    # Check if 'data.json' exists in the directory
    file_path = os.path.join(full_path, 'data.json')
    if os.path.isfile(file_path):
        # Edit the json file
        with open(file_path, 'r') as f:
            data = json.load(f)
            print(data['M'])

  0%|          | 0/10 [00:00<?, ?it/s]

5626
75
75


KeyboardInterrupt: 

In [3]:
def submit_mcmc_slurm(mcmc_common_dir, slurm_file):
    # List all the directories in the MCMC common directory
    # Only return directories that contain data.json
    dirs = []

    # List all items in the current directory
    for item in os.listdir(mcmc_common_dir):
        # Get the full path of the item
        full_path = os.path.join(mcmc_common_dir, item)
        
        # Check if the item is a directory
        # And if the directory contains data.json
        # Check if 'data.json' exists in the directory
        file_path = os.path.join(full_path, 'data.json')
        if os.path.isfile(file_path):
            dirs.append(full_path)
            
    # Submit a slurm job for each directory
    for dir in dirs:
        # Get the name of the directory
        
        dir_name = os.path.basename(dir)
        print(f"Submitting slurm job for {dir_name}")
        
        run_mcmc_py = '/mnt/home/tudomlumleart/ceph/01_ChromatinEnsembleRefinement/chromatin-ensemble-refinement/scripts/_run_mcmc.py'
        
        # Submit a slurm job
        cmd = f'sbatch {slurm_file} {run_mcmc_py} {dir}'
        
        # Run the command
        os.system(cmd)

In [4]:
slurm_file = '/mnt/home/tudomlumleart/ceph/01_ChromatinEnsembleRefinement/chromatin-ensemble-refinement/scripts/slurm/2024_RunPythonScript.sh'

In [6]:
submit_mcmc_slurm(mcmc_path, slurm_file)

Submitting slurm job for ctcfHox
Submitted batch job 4034882
Submitting slurm job for radNtHox
Submitted batch job 4034883
Submitting slurm job for radSox
Submitted batch job 4034884
Submitting slurm job for ctcfNtHox
Submitted batch job 4034885
Submitting slurm job for radHox
Submitted batch job 4034886
Submitting slurm job for ntHox
Submitted batch job 4034887
Submitting slurm job for ctcfNtSox
Submitted batch job 4034888
Submitting slurm job for radNtSox
Submitted batch job 4034889
Submitting slurm job for ctcfSox
Submitted batch job 4034890
Submitting slurm job for ntSox
Submitted batch job 4034891


In [ ]:
# write a function that runs MCMC on a data in the directory
def run_mcmc(mcmc_dir):
    # Load stan model 
    my_model = CmdStanModel(
        stan_file='/mnt/home/tudomlumleart/ceph/01_ChromatinEnsembleRefinement/chromatin-ensemble-refinement/scripts/stan/20240715_WeightOptimization.stan',
        cpp_options = {
            "STAN_THREADS": True,
        }
    )
    
    n_cores = multiprocessing.cpu_count()
    print(f"Number of CPU cores: {n_cores}")
    parallel_chains = 4
    threads_per_chain = int(n_cores / parallel_chains)
    print(f"Number of threads per chain: {threads_per_chain}")
    
    json_filename = os.path.join(mcmc_dir, 'data.json')
    stan_output_dir = os.path.join(mcmc_dir, 'stan_output')
    
    fit = my_model.sample(
        data=json_filename,
        chains=4,
        sig_figs=8,
        parallel_chains=parallel_chains,
        threads_per_chain=threads_per_chain,
        iter_warmup=1000,
        iter_sampling=1000,
        show_console=True,
    )
    
    # Save Stan output, i.e., posterior samples, in CSV format, in a specified folder
    fit.save_csvfiles(dir=stan_output_dir)